In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [21]:
from torch.utils.data import DataLoader,Dataset
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

import numpy as np
import pandas as pd

from torch.utils import data
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR

import cv2

class Dataset(Dataset):#Inheritsfromtorch.utils.data.Dataset
    def __init__(self):
        #defaultdirectorywheredataisloaded
        self.filepath = "/kaggle/input/car-steering-angle-prediction/driving_dataset/"
        self.filenames = os.listdir(self.filepath)
        
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self,index):
        filename = self.filenames[index]
        img = cv2.imread(self.filepath+filename)
        #Resizingimagesto(32,32)
        if img is not None:
            resized = cv2.resize(img,(32,32),interpolation=cv2.INTER_AREA)
            #return theimageconvertedtoanumpyarrayitscorrespondingsteeringangle
            return torch.from_numpy(resized.transpose()).float(),torch.rand(1)

In [22]:
class ConvNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 24, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(24, 36, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(36, 48, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(48, 64, 1),
            nn.ELU(),
            nn.Conv2d(64, 64, 1),
            nn.Dropout(0.5)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=64, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=50),
            nn.ELU(),
            nn.Linear(in_features=50, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )
        
    def forward(self,x): 
        #x = x.view(x.size(0), 3, 70, 320)
        x = self.conv_layers(x)
        # print(output.shape)
        # output = output.view(output.size(0), -1)
        x = torch.flatten(x, start_dim=1, end_dim=-1)
        output = self.linear_layers(x)
        
        return output
    

In [24]:
train_data = Dataset()

train_dataset, test_dataset, validation_dataset = torch.utils.data.random_split(train_data, [31898, 10000, 3672])


net=ConvNet()

optimizer=torch.optim.Adam(net.parameters(),lr=1e-3)
criterion=torch.nn.MSELoss()

train_loss = 0
iterr = 0
for i,sample_batched in enumerate(train_dataset):
    optimizer.zero_grad()
    if sample_batched is not None:
    
        inputs, angles = sample_batched
        inputs = inputs.unsqueeze(0)

        yhat=net(inputs)

        loss=criterion(yhat.squeeze(),torch.rand(1))

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        iterr = iterr + 1
    
print(f'Training Loss: {train_loss / iterr}')

In [25]:
test_loss = 0.0
iterr = 0
for i, sample_batched in enumerate(test_dataset):
    
    inputs, labels = sample_batched
    inputs = inputs.unsqueeze(0)
    
    yhat = net(inputs)
    loss = torch.sqrt(criterion(yhat.squeeze(), torch.rand(1)))
    
    
    test_loss += loss.item()
    iterr = iterr + 1
print(f'test Loss: {test_loss / iterr}')

## Configure search space for hyper-parameter tuning

In [27]:
from ray import tune
config = {
    "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}

In [ ]:
scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=5,
        grace_period=1,
        reduction_factor=2)

    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)